In [ ]:
!pip install openai

import pandas as pd
import re

from tqdm.notebook import tqdm
from openai import OpenAI
import json
import numpy as np

from sklearn.model_selection import train_test_split
from zipfile import ZipFile

SECRET_KEY = "//"

Link to presentation: https://docs.google.com/presentation/d/1M6HgvbykI0quBHufi2bbAuc-zPng1tRUZsNubz2e54Q/edit#slide=id.p

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')
# !ls /content/drive/MyDrive/*.csv.zip


GOOGLE_SPREADSHEET_LINK = "https://docs.google.com/spreadsheets/d/16m7l1DhlXFn0Rogicj-yezGmCrVsZlKEQyb2dW6OXFU/edit?usp=sharing"

sheet_id = GOOGLE_SPREADSHEET_LINK.split("/")[5]
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid=0"

dataset = pd.read_csv(csv_url)
dataset.head()

,Title,Text,Score
0,100% better sleep,I’ve been supporting Oura since it was a kicks...,5
1,One month in! It's great!,"I've been wearing the oura ring gen 4, stealth...",5
2,"Quite useful, despite some quirks",I bought the Oura 4 primarily to analyze my sl...,5
3,First time Full-size Foldable owner!,I originally got a Zflip 5 and liked the compa...,5
4,"For those ALREADY used to Samsung's ""Note/Gal...",After years of our family using a variety of A...,5


In [ ]:
dataset = dataset[(~dataset["Text"].isnull()) & (~dataset["Score"].isnull())] #  get rid of rows with a null score or null text

# create a subsample of the data
num_reviews_per_score = 20 # 10 test, 10 train
partial_dataframes = []
for score in [1, 2, 3, 4, 5]:
  reviews_for_score = dataset[dataset["Score"] == score]
  reviews_for_score = reviews_for_score.sample(n=num_reviews_per_score, random_state=48)
  partial_dataframes.append(reviews_for_score)
dataset = pd.concat(partial_dataframes, ignore_index=True)

dataset_train, dataset_test = train_test_split(dataset, test_size=0.5, random_state=48, shuffle=True, stratify=dataset["Score"].values)
print(len(dataset_train), "train examples")
print(len(dataset_test), "test examples")

dataset

50 train examples
50 test examples


,Title,Text,Score
0,A years experience with one clear winner! Its ...,"I have never such an indebt review, but I spen...",1
1,Picture Quality sucks & App is TERRIBLE!!,Picture quality on this sucks! I have my camer...,1
2,Buyer beware! Camera does NOT act as advertised,Camera works ok in wifi mode. I did have diffi...,1
3,basic camera features are locked behind monthl...,It's a shame because this WYZE Cam v4 itself s...,1
4,\nScam subscription clickbait trash,"My previous experience with Wyze ""smart"" door ...",1
...,...,...,...
95,I am a Hearing Impaired Customer- I CAN NOW HE...,The Beats Studio Pro Wireless Bluetooth Noise-...,5
96,The Beat Goes On,Beats headphones are a solid choice for anyone...,5
97,"For those ALREADY used to Samsung's ""Note/Gal...",After years of our family using a variety of A...,5
98,"If you can get it at a discount, it's a no bra...",Been using these for a few weeks now. I'm very...,5


In [ ]:
PROMPT_TEMPLATE = f"""
You are a review rating assistant trained to evaluate customer feedback and assign a review score between 1 and 5 to indicate user's setiment regarding the product they bought. Carefully analyze the review’s tone, word choice, and specificity. Use the scale below:

Carefully analyze the tone, strength of language, clarity of opinion, and presence of emotion. Use this consistent scoring rubric:
	•	1 – Very Negative: Clearly angry or very dissatisfied. Strong criticism or words like “terrible,” “worst,” “broken,” “never again.”
	•	2 – Negative: Mostly negative but less intense. May include phrases like “disappointed,” “not good,” “didn’t work well.”
	•	3 – Neutral: Mixed or vague tone. Balanced pros and cons. Words like “okay,” “fine,” or just factual descriptions.
	•	4 – Positive: Generally positive with some minor issues. Words like “good,” “liked it,” “satisfied.”
	•	5 – Very Positive: Highly enthusiastic and full of praise. Words like “amazing,” “perfect,” “highly recommend,” “love it.”


{{product}}
Task:

1 Star example:
Title: Bad Keyboard, Even Worse Customer Service
Content: The one star is as much for Amazon's presentation and returns policy with respect to this keyboard as it is about the keyboard itself.

In need of replacing my long-serving and excellent Apple wired keyboard, I wanted a new keyboard as close to the original as possible, in terms of layout, look, functionality and reliability. Unfortunately, after much time-consuming exploration, the only candidate I could find that came even close to matching my criteria was this Apple so-called "Magic" Keyboard. I finally pulled the trigger on purchasing it after asking Rufus, Amazon's supposedly genius new AI assistant, if the Magic Keyboard could be used as a wired keyboard with Bluetooth turned off, and Rufus answered that it indeed could.

Well, sadly, after receiving the keyboard and trying it out, I quickly discovered that Rufus was dead wrong in his answer to my question (though in fairness, Rufus has correctly and even intuitively answered other questions that I've posed about other items).

So, with that strike against the keyboard, plus the fact that its pointlessly light gray letters on the keys are annoying at best and likely prone to fading in the future at worst, and also the fact that the Magic Keyboard, unlike the old Apple wired keyboard, lays flat with no riser underneath the front to lift and curve it slightly — in view of all of these negatives, I decided to return this piece of ... (ahem) "Magic.”

And that's where Amazon REALLY fails big-time on this item. For, as a few other reviewers have noted, the only way one can return this Apple product is IN-PERSON to a Whole Foods or an Amazon store. No UPS, no USPS, no other way. Yes, despite the fact that you can order it online and Amazon delivers it to you, you have to return it IN-PERSON!

I complained to Amazon Customer Service about this — and even offered to pay for the UPS return myself — and they still insisted that it had to be returned IN-PERSON!! Had I known this in advance, I NEVER would have purchase the keyboard from Amazon. But — and here's where Amazon's return policy on this item is not just a nuisance, but also highly deceptive — there is absolutely no mention in Amazon's stated return policy pertaining to this keyboard that the item can only be returned IN-PERSON to a Whole Foods or Amazon store. That being the case, I've filed a complaint against Amazon with the New York State Division of Consumer Protection. If Amazon is going to maintain such a return policy, the restrictive terms about “in-person returns only” should be spelled out in writing. Currently, they are not.

Explanation: There are many negative words in this comment like "dead wrong", "fail", "pointlessly", "annoying". The "REALLY" is in caps to strengthen the negative feeling. Overall the review is strongly negative, and thus is a 1-star review.

2 Star example:
Title: keys do NOT last!
Content: Bought my first one and it lasted just under a year...the L side shift key became sticky and randomly typed capital letters. I cleaned it, researched and popped key off, got it to work another month or so before it gave up completely. I was told "practically disposable" and not worth cost of repair. But I was very happy with the feeling, use, and connectivity to my MacBook. Yes. I bought another.
This next keyboard has only lasted 4 months and the L shift key is again sticking. I have been very conciencious about whether I lean on that key etc, and I don't! I haven't spilt or eaten on it. again popped off and cleaned, didn't see anything but it worked another couple weeks and started in again.
I've purchased another keyboard, cheaper but Apple compatible. Even if this new one only lasts a year, at least I will have only spent 1/3 of the price!
Very disappointed. I like the look, feel, function, quick connectivity, easy set up...but not worth the $ when it won't even last a year!!
Not worth the $ unfortunately.

Explanation: The title already shows negative view towards this. Overall the content is negative, not talking good to the product. However, it is of less intensity and is very sensible. So it is a 2-star example.

3 Star example:
Title: A Programmer's Frustration
Content: Oh, where do I even begin with the Apple Magic Keyboard with Numeric Keypad? As a weary programmer shackled to my desk, this keyboard has been a rollercoaster of emotions. It's like a bug-riddled code that just won't compile correctly - frustrating and time-consuming.

Let's start with the good, if you can call it that. The battery life is like a glimmer of hope in the dark abyss of my programming struggles. It somehow manages to outlast my never-ending coding sessions, and for that, I give it a nod of begrudging approval. The Bluetooth connectivity is also surprisingly stable, much unlike my mental state after hours of debugging.

But alas, the typing experience is where this magical journey takes a turn. At first, I marveled at the soft, whisper-quiet keystrokes that were supposed to make my typing experience heavenly. But soon enough, the novelty wore off, and I realized that these keys are like spongy marshmallows. My fingers sink into them, and I'm left with this strange feeling of typing on a cloud. It might be great for some, but for a programmer like me who demands precision and speed, it's like trying to type in molasses.

And don't get me started on the numeric keypad. As someone who relies on numbers as much as code syntax, I expected this to be a game-changer. Instead, it's more like a never-ending test of my patience. The keys are cramped, and the layout is just off, making me feel like I'm navigating through a labyrinth of numeric confusion.

And let's not forget the L and O keys, the thorns in my coding journey. After only a few months, these two troublemakers decided to revolt and popped off. No matter how much I fiddled and cursed, they refused to go back to their rightful places. It's like they staged a rebellion against my programming efforts, leaving me in a constant state of annoyance.

In conclusion, the Apple Magic Keyboard with Numeric Keypad is a mixed bag of emotions for this disgruntled programmer. While the battery life and Bluetooth connectivity offer a glimmer of hope, the spongy keys and cramped numeric keypad have tested my endurance. The rebellious L and O keys were the final straw, earning this keyboard three stars from me. It's just not the coding companion I had hoped for, and I'll be seeking a more developer-friendly keyboard to rescue me from this techno-quicksand.

Explanation: The title shows some frustration, which is negative. In the content, there are words that indicate mixed experience: "rollercoaster", "mixed bag of emotions". Also the user identifies a good spot. Making it a relatively neutral 3-star review.

4 Star example:
Title: Key Placement
Content: Just be mindful of the standard placement of the right side command button and the bottom left corner function button. They don't line up with the computer's placement, and if you've built the muscle memory with your finger or thumb and use these button a lot it will throw you off. Seriously, what sort of "magic" is this keyboard?
Super annoying, took a while before I readjusted, and now when I use the built-in keyboard while on the go I have to flip that switch in my brain of which one I'm using.
Otherwise, this product is sleek and I like it a lot. Specifically, the additional buttons between the primary key set and the numpad are clutch. The home and end, page up and down, and the delete key [—>] that removes a character in the opposite direction of the backspace [<—]s great. It's that stupid command key that can screws me up because of muscle memory with my thumb.
Lastly, maybe there's a good reason they adjusted the command and function key placement. But if you're familiar with Apple's MO, it's just another reminder of their product gimicks—always changing things that don't need it. It's like no one ever told Steve Jobs and the crew about the phrase, "If it ain't broke, don't fix it." Or maybe they misheard and just went with, "If it ain't broke, let's change it anyway so we can make more money and screw over our faithful customer base who'll buy our elitist products anyway."
This assessment is going to come off hypocritical because I obviously have a macbook, but it's my work computer.

Explanation: the user expressed that he likes the product overall, however, at the same time it points out some defectives. However, the user didn't emphasize this, retaining a mostly postive emotion. This makes it a 4-star review.

5 Star example:
Title: A Game Changer for My Workspace!
Content: I’ve been using the Apple Magic Keyboard with Numeric Keypad for a few weeks now, and I’m honestly blown away by how much it has improved my typing experience. As someone who works on both my Mac and iPad regularly, the wireless Bluetooth connection makes it super easy to switch between devices. No more fumbling with wires or constantly having to reconnect—it just works.

The keys are smooth, quiet, and responsive, which has made typing a breeze, even during long sessions. The additional numeric keypad is a huge bonus for anyone who does a lot of data entry or works with numbers—it really speeds things up.

The rechargeable battery is fantastic, and I love not having to worry about swapping out batteries all the time. The build quality is solid, and as with most Apple products, it just feels premium and sleek. It also pairs effortlessly with my iPhone when I need to use it on the go.

Overall, this keyboard has been an amazing addition to my setup. If you're looking for a reliable, high-quality wireless keyboard for your Apple devices, I highly recommend this one!

Explanation: The title is intensively positive. The content is nearly all praises with words like "Reliable", "smooth, quiet, responsive", "huge bonus". The user recommends the product in the end. The strong positive emotion makes it a 5-star review.

Now use your own reasoning, analyze the review, try to find hints that indicates the person's sentiments and the corresponding rating. End with a single number from at least 1 to at most 5. You can have some decimal places, but an integer is preferred.

Review Title: {{summary}}
Review Text: {{text}}

 """

def apply_prompt_template(template, summary, text, product=""):
	if product != "":
		product = f"Review on this product:{product}"

	return template.format(summary=summary, text=text, product=product)

dataset_train["prompt"] = dataset_train.apply(lambda x: apply_prompt_template(PROMPT_TEMPLATE, x["Title"], x["Text"]), axis=1)
dataset_test["prompt"] = dataset_test.apply(lambda x: apply_prompt_template(PROMPT_TEMPLATE, x["Title"], x["Text"]), axis=1)

for prompt in dataset_test["prompt"][:5]:
  print("Prompt:\n")
  print(prompt)
  print()

Prompt:


You are a review rating assistant trained to evaluate customer feedback and assign a review score between 1 and 5 to indicate user's setiment regarding the product they bought. Carefully analyze the review’s tone, word choice, and specificity. Use the scale below:

Carefully analyze the tone, strength of language, clarity of opinion, and presence of emotion. Use this consistent scoring rubric:
	•	1 – Very Negative: Clearly angry or very dissatisfied. Strong criticism or words like “terrible,” “worst,” “broken,” “never again.”
	•	2 – Negative: Mostly negative but less intense. May include phrases like “disappointed,” “not good,” “didn’t work well.”
	•	3 – Neutral: Mixed or vague tone. Balanced pros and cons. Words like “okay,” “fine,” or just factual descriptions.
	•	4 – Positive: Generally positive with some minor issues. Words like “good,” “liked it,” “satisfied.”
	•	5 – Very Positive: Highly enthusiastic and full of praise. Words like “amazing,” “perfect,” “highly recommend

In [ ]:
client = OpenAI(api_key=SECRET_KEY)

num_test = 50
gpt_responses = []
for prompt in tqdm(dataset_test["prompt"].values[:num_test], total=num_test):

  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        # { "role": "system", "content": [{"type": "text", "text": ""}]},
        { "role": "user", "content": [{"type": "text", "text":  prompt}]}
      ],
      temperature=1,
      # max_tokens=1,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      # logprobs=True,
      # top_logprobs=1,
    )

  gpt_output = completion.choices[0].message.content
  gpt_responses.append(gpt_output)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Convert GPT responses to numbers
gpt_responses_num = np.array([float(re.findall(r'\d+(?:\.\d+)?', response)[-1]) for response in gpt_responses])
correct_ratings = dataset_test["Score"].values[:num_test]

# Create DataFrame
results_df = pd.DataFrame({
    "Correct Rating": correct_ratings,
    "GPT Rating": gpt_responses_num,
    "Off": np.abs(gpt_responses_num - correct_ratings)
})

# Calculate metrics
errors = [np.abs(gpt_responses_num[i] - correct_ratings[i]) for i in range(len(correct_ratings))]
mean_absolute_error = np.mean(errors)

errors_sq = [(gpt_responses_num[i] - correct_ratings[i])**2 for i in range(len(correct_ratings))]
rmse = np.sqrt(np.mean(errors))

# Print metrics
print(f"Mean Absolute Error: {mean_absolute_error}")
print(f"Root Mean Squared Error: {rmse}")
print(results_df)

# Possible Question: what's the difference between rmse and mae?

Mean Absolute Error: 0.53
Root Mean Squared Error: 0.7280109889280518
    Correct Rating  GPT Rating  Off
0                1         1.0  0.0
1                4         3.0  1.0
2                3         3.0  0.0
3                2         4.0  2.0
4                4         2.0  2.0
5                4         4.0  0.0
6                4         3.0  1.0
7                5         4.0  1.0
8                3         3.0  0.0
9                5         4.0  1.0
10               4         4.0  0.0
11               4         4.0  0.0
12               5         5.0  0.0
13               3         2.0  1.0
14               1         4.5  3.5
15               1         1.0  0.0
16               5         5.0  0.0
17               4         4.0  0.0
18               1         2.0  1.0
19               2         2.0  0.0
20               1         1.0  0.0
21               5         5.0  0.0
22               2         1.0  1.0
23               4         4.0  0.0
24               1         1.0

In [ ]:
# For plugging into random texts we found on social media
# Business value: unify different score scales, quantify social media reviews that don't have values. Help quantify user words to help with product evaluation.
client = OpenAI(api_key=SECRET_KEY)

num_test = 5
gpt_responses = []
title = "Great feel for me on the keyboard"
text = """I like that the LED display is controlled by the keyboard and not problematic software. The less software I need running the better. I bought this as an impulse buy one day while doing shopping at Walmart. I felt a few keys through the packaging and they felt nice to me. I got it home and decided I loved the feel of the keys. I like hearing the clanking sound that tells me each key has been pushed. The waving illumination is pretty to look at, but the feel of the keyboard is dynamite. I went to buy another at Walmart a month later for another computer at home and it was not longer sold. I bought it by ordering it on their website. I am glad I did. It is inexpensive and has a great feel for me."""
# product = " " + "keyboard"
prompt = apply_prompt_template(PROMPT_TEMPLATE, title, text, product)
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      # { "role": "system", "content": [{"type": "text", "text": ""}]},
      { "role": "user", "content": [{"type": "text", "text":  prompt}]}
    ],
    temperature=1,
    # max_tokens=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    # logprobs=True,
    # top_logprobs=1,
  )

gpt_output = completion.choices[0].message.content
gpt_responses.append(gpt_output)
gpt_responses_num = np.array([float(re.findall(r'\d+(?:\.\d+)?', response)[-1]) for response in gpt_responses])
print("GPT ratings: ", gpt_responses_num)

GPT ratings:  [5.]


In [ ]:
# For plugging into random texts we found on social media
# Business value: unify different score scales, quantify social media reviews that don't have values. Help quantify user words to help with product evaluation.
client = OpenAI(api_key=SECRET_KEY)

num_test = 5
gpt_responses = []
title = ""
text = """This sucks"""
product = ""
prompt = apply_prompt_template(PROMPT_TEMPLATE, title, text, product)
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      # { "role": "system", "content": [{"type": "text", "text": ""}]},
      { "role": "user", "content": [{"type": "text", "text":  prompt}]}
    ],
    temperature=1,
    # max_tokens=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    # logprobs=True,
    # top_logprobs=1,
  )

gpt_output = completion.choices[0].message.content
gpt_responses.append(gpt_output)
gpt_responses_num = np.array([float(re.findall(r'\d+(?:\.\d+)?', response)[-1]) for response in gpt_responses])
print("GPT ratings: ", gpt_responses_num)

GPT ratings:  [1.]
